In [6]:
import skrf as rf
from math import pi
from skrf.media.cpw import CPW
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from modeling.utils.resonators import cpw_cl_ll, cpw_capacitance_inductance, get_equivalent_lc, resonator_kappa
from modeling.utils.network import assemble_network
from modeling.utils.qubits import *
from qucat import Network,L,J,C,R
from modeling.clockmon.libraries.clockmon_library import clockmon_coupling_libraries, clockmon_library, clockmon_cqr_to_ground
from modeling.smooth_capacitor.libraries.smooth_capacitor_library import digit_tee_library, digit_tee_ck_to_ground, get_ck

In [7]:
smooth_capacitor_lib = digit_tee_library(deembed = [300, 200])
ck_to_ground = digit_tee_ck_to_ground(deembed=[300, 200])
clockmon_lib = clockmon_library(deembed = 200)
cqr_to_ground = clockmon_cqr_to_ground(deembed = 200)

In [8]:
planck_h = 6.62607015e-34
e_charge = 1.60217663e-19
phi0 =  2.067833848e-15 # Wb

a = 10e-6
b = 6e-6
ep_r = 11.9

nb_qubits = 4
rr_freq_target = 7e9
c_sigma_unloaded = 90e-15
kappa_target = 1e6
target_parameters = {
    "chi": 0.5e6,
    "qubit_frequency": 4.2e9,
    "rr_frequencies_target": rr_freq_target,
    "c_sigma_unloaded": c_sigma_unloaded
}
Z0 = CPW(w = a, s = b, ep_r = ep_r).z0[0].real
l_r = Z0*pi/(4*pi*rr_freq_target)
c_r = 1/((2*pi*rr_freq_target)**2*l_r)

In [9]:
# c_qr = 1.653523031555808e-15
# l_j = 13e-9
# cir = Network([
#     C(0,1,get_csigma_cqr(cmatrix)[0] -c_qr), # transmon
#     J(0,1,'Lj'),
#     C(0,2,c_r), # resonator
#     L(0,2,l_r),
#     C(1,2,c_qr), # coupling capacitor
# ])
# err_chi = abs(cir.kerr(Lj = l_j)[0,1] - target_parameters["chi"])

# H = cir.hamiltonian(modes = [0, 1],
#                     taylor = 12,
#                     excitations = [10, 10],
#                     Lj = l_j)
# ee = H.eigenenergies().real
# ee -= ee[0]
# print("qubit freq:", ee[1])

In [10]:
from scqubits import Transmon

In [11]:
Ic = get_Ic_from_Lj(l_j)
EC = e_charge**2/(c_sigma_unloaded)/2/planck_h
EC_unloaded = e_charge**2/(c_sigma_unloaded)/2/planck_h
Ej = Ic*phi0/2/pi/planck_h

13278758142.465513

In [12]:
transmon = Transmon(EJ = Ej, EC=EC, ng = 0, ncut = 31)
transmon.E01()

4555271983.166996

In [13]:
cmatrix = clockmon_lib(20)
l_j = 13e-9
cir = Network([
    C(0, 1, cmatrix[1,1]), # island1 to ground
    C(0, 2, cmatrix[2,2]), #island2 to ground
    L(1, 2, "Lj"), # jj
    C(1, 2, cmatrix[1,2]), # island1 to island2
    C(1, 3, cmatrix[0, 1]), # island1 to resonator
    C(2, 3, cmatrix[0, 2]), # island2 to resonator
    C(0,3,c_r), # resonator
    L(0,3,l_r),
])
err_chi = abs(cir.kerr(Lj = l_j)[0,1] - target_parameters["chi"])

H = cir.hamiltonian(modes = [0, 1],
                    taylor = 12,
                    excitations = [10, 10],
                    Lj = l_j)
ee = H.eigenenergies().real
ee -= ee[0]
print("qubit freq:", ee[1])
print("Cqr:", get_csigma_cqr(cmatrix)[1])

qubit freq: 4671245488.773653
Cqr: 1.653523031555808e-15


In [14]:
l_j = 13e-9
cir = Network([
    C(0, 1, cmatrix[1,1]), # island1 to ground
    C(0, 2, cmatrix[2,2]), #island2 to ground
    L(1, 2, "Lj"), # jj
    C(1, 2, cmatrix[1,2]), # island1 to island2
    C(1, 3, cmatrix[0, 1]), # island1 to resonator
    C(2, 3, cmatrix[0, 2]), # island2 to resonator
    C(0,3,c_r), # resonator
    L(0,3,l_r),
])
err_chi = abs(cir.kerr(Lj = l_j)[0,1] - target_parameters["chi"])

H = cir.hamiltonian(modes = [0, 1],
                    taylor = 12,
                    excitations = [10, 10],
                    Lj = l_j)
ee = H.eigenenergies().real
ee -= ee[0]
print("qubit freq:", ee[1])
print("Cqr:", get_csigma_cqr(cmatrix)[1])

qubit freq: 4671245488.773653
Cqr: 1.653523031555808e-15


In [15]:
get_csigma_cqr(cmatrix)[0] #+ c_qr


8.928824039566648e-14

In [16]:
cmatrix[1,2] + (1/ (cmatrix[1,1] + cmatrix[0, 1]) + 1 / (cmatrix[2,2] + cmatrix[0, 2]))**(-1)

8.928824039566648e-14

In [17]:
c_l, l_l = cpw_cl_ll(a, b, ep_r)
resonator_waveguide_lengths = np.pi * np.sqrt((c_r * l_r) / (c_l*l_l))
c_l_pozar = c_l * resonator_waveguide_lengths
l_l_pozar = l_l * resonator_waveguide_lengths

In [18]:
cir = Network([
    C(0,3,c_r), # resonator
    L(0,3,l_r),
])

H = cir.hamiltonian(modes = [0],
                    taylor = 12,
                    excitations = [10],)
ee = H.eigenenergies().real
ee -= ee[0]
print("qubit freq:", ee[1])

qubit freq: 7000000000.000009
